In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile0 = pd.read_pickle("BBB_Train_Decile0.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile0.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile0, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 0')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 2s 566us/step - loss: 0.6182 - acc: 0.6735 - val_loss: 0.5825 - val_acc: 0.6969
Epoch 35/75
3087/3087 [==============================] - 2s 521us/step - loss: 0.6158 - acc: 0.6718 - val_loss: 0.5825 - val_acc: 0.6969
Epoch 36/75
3087/3087 [==============================] - 2s 568us/step - loss: 0.6195 - acc: 0.6718 - val_loss: 0.5825 - val_acc: 0.6969
Epoch 37/75
3087/3087 [==============================] - 2s 623us/step - loss: 0.6152 - acc: 0.6722 - val_loss: 0.5826 - val_acc: 0.6969
Epoch 38/75
3087/3087 [==============================] - 2s 530us/step - loss: 0.6120 - acc: 0.6728 - val_loss: 0.5825 - val_acc: 0.6969
Epoch 39/75
3087/3087 [==============================] - 2s 543us/step - loss: 0.6157 - acc: 0.6725 - val_loss: 0.5826 - val_acc: 0.6969
Epoch 40/75
3087/3087 [==============================] - 2s 588us/step - loss: 0.6184 - acc: 0.6715 - val_loss: 0.5824 - val_acc: 0.6969
Epoch 41/75
3087/3087 [==============================

3087/3087 [==============================] - 2s 571us/step - loss: 0.6141 - acc: 0.6754 - val_loss: 0.5802 - val_acc: 0.6956
Epoch 18/75
3087/3087 [==============================] - 2s 580us/step - loss: 0.6149 - acc: 0.6751 - val_loss: 0.5803 - val_acc: 0.6956
Epoch 19/75
3087/3087 [==============================] - 2s 561us/step - loss: 0.6106 - acc: 0.6731 - val_loss: 0.5804 - val_acc: 0.6943
Epoch 20/75
3087/3087 [==============================] - 2s 572us/step - loss: 0.6122 - acc: 0.6754 - val_loss: 0.5804 - val_acc: 0.6943

Epoch 00020: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 21/75
3087/3087 [==============================] - 2s 518us/step - loss: 0.6119 - acc: 0.6748 - val_loss: 0.5803 - val_acc: 0.6956
Epoch 22/75
3087/3087 [==============================] - 2s 572us/step - loss: 0.6099 - acc: 0.6757 - val_loss: 0.5803 - val_acc: 0.6969
Epoch 23/75
3087/3087 [==============================] - 2s 529us/step - loss: 0.6127 - acc: 0.6751 - val_loss: 0.5803 - val_

3087/3087 [==============================] - 4s 1ms/step - loss: 0.6984 - acc: 0.5653 - val_loss: 0.6338 - val_acc: 0.6982
Epoch 2/100
3087/3087 [==============================] - 2s 559us/step - loss: 0.6282 - acc: 0.6702 - val_loss: 0.5893 - val_acc: 0.6982
Epoch 3/100
3087/3087 [==============================] - 2s 652us/step - loss: 0.6290 - acc: 0.6696 - val_loss: 0.5860 - val_acc: 0.6982
Epoch 4/100
3087/3087 [==============================] - 2s 600us/step - loss: 0.6201 - acc: 0.6725 - val_loss: 0.5850 - val_acc: 0.6982
Epoch 5/100
3087/3087 [==============================] - 2s 534us/step - loss: 0.6166 - acc: 0.6722 - val_loss: 0.5829 - val_acc: 0.6982
Epoch 6/100
3087/3087 [==============================] - 2s 604us/step - loss: 0.6161 - acc: 0.6715 - val_loss: 0.5835 - val_acc: 0.6995
Epoch 7/100
3087/3087 [==============================] - 1s 477us/step - loss: 0.6134 - acc: 0.6731 - val_loss: 0.5820 - val_acc: 0.6995
Epoch 8/100
3087/3087 [==============================] 

3087/3087 [==============================] - 1s 435us/step - loss: 0.6127 - acc: 0.6744 - val_loss: 0.5818 - val_acc: 0.6982
Epoch 60/100
3087/3087 [==============================] - 1s 453us/step - loss: 0.6163 - acc: 0.6738 - val_loss: 0.5819 - val_acc: 0.6982
Epoch 61/100
3087/3087 [==============================] - 1s 422us/step - loss: 0.6142 - acc: 0.6718 - val_loss: 0.5818 - val_acc: 0.6982
Epoch 62/100
3087/3087 [==============================] - 1s 393us/step - loss: 0.6125 - acc: 0.6741 - val_loss: 0.5818 - val_acc: 0.6982
Epoch 63/100
3087/3087 [==============================] - 1s 388us/step - loss: 0.6149 - acc: 0.6744 - val_loss: 0.5818 - val_acc: 0.6982
Epoch 64/100
3087/3087 [==============================] - 1s 376us/step - loss: 0.6135 - acc: 0.6748 - val_loss: 0.5818 - val_acc: 0.6982
Epoch 65/100
3087/3087 [==============================] - 1s 386us/step - loss: 0.6091 - acc: 0.6715 - val_loss: 0.5817 - val_acc: 0.6982
Epoch 66/100
3087/3087 [=======================

...

3087/3087 [==============================] - 8s 3ms/step - loss: 0.6535 - acc: 0.6511 - val_loss: 0.5915 - val_acc: 0.6878
Epoch 2/125
3087/3087 [==============================] - 2s 607us/step - loss: 0.6177 - acc: 0.6715 - val_loss: 0.5851 - val_acc: 0.6878
Epoch 3/125
3087/3087 [==============================] - 2s 630us/step - loss: 0.6196 - acc: 0.6741 - val_loss: 0.5845 - val_acc: 0.6878
Epoch 4/125
3087/3087 [==============================] - 2s 612us/step - loss: 0.6129 - acc: 0.6718 - val_loss: 0.5812 - val_acc: 0.6943
Epoch 5/125
3087/3087 [==============================] - 2s 658us/step - loss: 0.6158 - acc: 0.6748 - val_loss: 0.5833 - val_acc: 0.6969
Epoch 6/125
3087/3087 [==============================] - 2s 683us/step - loss: 0.6170 - acc: 0.6725 - val_loss: 0.5827 - val_acc: 0.6969
Epoch 7/125
3087/3087 [==============================] - 2s 581us/step - loss: 0.6117 - acc: 0.6744 - val_loss: 0.5809 - val_acc: 0.6969
Epoch 8/125
3087/3087 [==============================] 

3087/3087 [==============================] - 1s 366us/step - loss: 0.6183 - acc: 0.6715 - val_loss: 0.5833 - val_acc: 0.6969
Epoch 98/100
3087/3087 [==============================] - 1s 382us/step - loss: 0.6168 - acc: 0.6748 - val_loss: 0.5833 - val_acc: 0.6969
Epoch 99/100
3087/3087 [==============================] - 1s 402us/step - loss: 0.6165 - acc: 0.6709 - val_loss: 0.5834 - val_acc: 0.6969
Epoch 100/100
3087/3087 [==============================] - 2s 511us/step - loss: 0.6145 - acc: 0.6728 - val_loss: 0.5834 - val_acc: 0.6969
Test accuracy: 0.69689119171
Train on 3087 samples, validate on 772 samples
Epoch 1/50
3087/3087 [==============================] - 9s 3ms/step - loss: 0.6535 - acc: 0.6485 - val_loss: 0.5961 - val_acc: 0.6904
Epoch 2/50
3087/3087 [==============================] - 2s 554us/step - loss: 0.6193 - acc: 0.6709 - val_loss: 0.5908 - val_acc: 0.6891
Epoch 3/50
3087/3087 [==============================] - 2s 564us/step - loss: 0.6178 - acc: 0.6722 - val_loss: 0.5

3087/3087 [==============================] - 1s 378us/step - loss: 0.6155 - acc: 0.6741 - val_loss: 0.5803 - val_acc: 0.6982
Epoch 31/100
3087/3087 [==============================] - 1s 373us/step - loss: 0.6167 - acc: 0.6725 - val_loss: 0.5802 - val_acc: 0.6982
Epoch 32/100
3087/3087 [==============================] - 1s 367us/step - loss: 0.6158 - acc: 0.6699 - val_loss: 0.5802 - val_acc: 0.6982
Epoch 33/100
3087/3087 [==============================] - 1s 379us/step - loss: 0.6096 - acc: 0.6741 - val_loss: 0.5802 - val_acc: 0.6982
Epoch 34/100
3087/3087 [==============================] - 1s 386us/step - loss: 0.6130 - acc: 0.6712 - val_loss: 0.5803 - val_acc: 0.6982
Epoch 35/100
3087/3087 [==============================] - 1s 381us/step - loss: 0.6133 - acc: 0.6735 - val_loss: 0.5802 - val_acc: 0.6982
Epoch 36/100
3087/3087 [==============================] - 1s 383us/step - loss: 0.6154 - acc: 0.6718 - val_loss: 0.5802 - val_acc: 0.6982
Epoch 37/100
3087/3087 [=======================

3087/3087 [==============================] - 1s 226us/step - loss: 0.6170 - acc: 0.6718 - val_loss: 0.5801 - val_acc: 0.6982
Epoch 90/100
3087/3087 [==============================] - 1s 227us/step - loss: 0.6120 - acc: 0.6728 - val_loss: 0.5800 - val_acc: 0.6982
Epoch 91/100
3087/3087 [==============================] - 1s 227us/step - loss: 0.6144 - acc: 0.6748 - val_loss: 0.5800 - val_acc: 0.6982
Epoch 92/100
3087/3087 [==============================] - 1s 230us/step - loss: 0.6187 - acc: 0.6725 - val_loss: 0.5800 - val_acc: 0.6982
Epoch 93/100
3087/3087 [==============================] - 1s 231us/step - loss: 0.6137 - acc: 0.6744 - val_loss: 0.5801 - val_acc: 0.6982
Epoch 94/100
3087/3087 [==============================] - 1s 229us/step - loss: 0.6098 - acc: 0.6731 - val_loss: 0.5801 - val_acc: 0.6982
Epoch 95/100
3087/3087 [==============================] - 1s 234us/step - loss: 0.6158 - acc: 0.6709 - val_loss: 0.5800 - val_acc: 0.6982
Epoch 96/100
3087/3087 [=======================

3087/3087 [==============================] - 1s 227us/step - loss: 0.6150 - acc: 0.6738 - val_loss: 0.5812 - val_acc: 0.6969
Epoch 48/50
3087/3087 [==============================] - 1s 228us/step - loss: 0.6104 - acc: 0.6767 - val_loss: 0.5811 - val_acc: 0.6969
Epoch 49/50
3087/3087 [==============================] - 1s 226us/step - loss: 0.6095 - acc: 0.6751 - val_loss: 0.5811 - val_acc: 0.6969
Epoch 50/50
3087/3087 [==============================] - 1s 227us/step - loss: 0.6123 - acc: 0.6738 - val_loss: 0.5811 - val_acc: 0.6969
Test accuracy: 0.69689119171
Evalutation of best performing model:
1521/1521 [==============================] - 0s 67us/step
[0.56055577760617403, 0.75805391201762917]
Best performing model chosen hyper-parameters:
{'epochs': 2, 'lr': 1}
Confusion Matrix for LSTM predictions:
[[   1  368]
 [   0 1152]]
Classification Report for LSTM predictions:
-------------------------------------------
             precision    recall  f1-score   support

          0       

In [ ]:
0.8297